# Генеративни мрежи

Рекурентните невронни мрежи (RNNs) и техните варианти с управлявани клетки, като клетки с дългосрочна памет (LSTMs) и управлявани рекурентни единици (GRUs), предоставят механизъм за моделиране на език, т.е. те могат да научат подредбата на думите и да предсказват следващата дума в последователност. Това ни позволява да използваме RNNs за **генеративни задачи**, като обикновено генериране на текст, машинен превод и дори създаване на описания за изображения.

В архитектурата на RNN, която обсъдихме в предишния модул, всяка RNN единица произвежда следващото скрито състояние като изход. Въпреки това, можем да добавим още един изход към всяка рекурентна единица, което би ни позволило да изведем **последователност** (която е равна по дължина на оригиналната последователност). Освен това, можем да използваме RNN единици, които не приемат вход на всяка стъпка, а просто вземат някакъв начален вектор на състояние и след това произвеждат последователност от изходи.

В този тетраден файл ще се фокусираме върху прости генеративни модели, които ни помагат да генерираме текст. За простота, нека изградим **мрежа на ниво символи**, която генерира текст буква по буква. По време на обучението трябва да вземем някакъв текстов корпус и да го разделим на последователности от букви.


In [1]:
import tensorflow as tf
from tensorflow import keras
import tensorflow_datasets as tfds
import numpy as np

ds_train, ds_test = tfds.load('ag_news_subset').values()

## Създаване на речник от символи

За да изградим генеративна мрежа на ниво символи, трябва да разделим текста на отделни символи вместо на думи. `TextVectorization` слой, който използвахме преди, не може да направи това, затова имаме две опции:

* Ръчно зареждане на текста и извършване на токенизация „на ръка“, както е показано в [този официален пример на Keras](https://keras.io/examples/generative/lstm_character_level_text_generation/)
* Използване на класа `Tokenizer` за токенизация на ниво символи.

Ще изберем втория вариант. `Tokenizer` може да се използва и за токенизация на ниво думи, така че лесно може да се премине от токенизация на ниво символи към токенизация на ниво думи.

За да извършим токенизация на ниво символи, трябва да предадем параметъра `char_level=True`:


In [2]:
def extract_text(x):
    return x['title']+' '+x['description']

def tupelize(x):
    return (extract_text(x),x['label'])

tokenizer = keras.preprocessing.text.Tokenizer(char_level=True,lower=False)
tokenizer.fit_on_texts([x['title'].numpy().decode('utf-8') for x in ds_train])

Искаме също да използваме един специален токен, за да обозначим **края на последователността**, който ще наречем `<eos>`. Нека го добавим ръчно към речника:


In [3]:
eos_token = len(tokenizer.word_index)+1
tokenizer.word_index['<eos>'] = eos_token

vocab_size = eos_token + 1

Сега, за да кодираме текст в последователности от числа, можем да използваме:


In [4]:
tokenizer.texts_to_sequences(['Hello, world!'])

[[48, 2, 10, 10, 5, 44, 1, 25, 5, 8, 10, 13, 78]]

## Обучение на генеративна RNN за създаване на заглавия

Начинът, по който ще обучим RNN да генерира новинарски заглавия, е следният. На всяка стъпка ще вземем едно заглавие, което ще бъде подадено в RNN, и за всеки входен символ ще поискаме от мрежата да генерира следващия изходен символ:

![Изображение, показващо пример за генериране на думата 'HELLO' с RNN.](../../../../../translated_images/bg/rnn-generate.56c54afb52f9781d.webp)

За последния символ от нашата последователност ще поискаме от мрежата да генерира токен `<eos>`.

Основната разлика при генеративната RNN, която използваме тук, е, че ще вземем изхода от всяка стъпка на RNN, а не само от последната клетка. Това може да се постигне чрез задаване на параметъра `return_sequences` на RNN клетката.

Така, по време на обучението, входът към мрежата ще бъде последователност от кодирани символи с определена дължина, а изходът ще бъде последователност със същата дължина, но изместена с един елемент и завършваща с `<eos>`. Минипакетът ще се състои от няколко такива последователности, и ще трябва да използваме **допълване** (padding), за да изравним всички последователности.

Нека създадем функции, които ще трансформират набора от данни за нас. Тъй като искаме да допълваме последователностите на ниво минипакет, първо ще групираме набора от данни, като извикаме `.batch()`, а след това ще използваме `map`, за да извършим трансформацията. Следователно, функцията за трансформация ще приема цял минипакет като параметър:


In [5]:
def title_batch(x):
    x = [t.numpy().decode('utf-8') for t in x]
    z = tokenizer.texts_to_sequences(x)
    z = tf.keras.preprocessing.sequence.pad_sequences(z)
    return tf.one_hot(z,vocab_size), tf.one_hot(tf.concat([z[:,1:],tf.constant(eos_token,shape=(len(z),1))],axis=1),vocab_size)

Няколко важни неща, които правим тук:
* Първо извличаме действителния текст от низовия тензор
* `text_to_sequences` преобразува списъка от низове в списък от цели тензори
* `pad_sequences` допълва тези тензори до тяхната максимална дължина
* Накрая извършваме еднократно кодиране на всички символи, както и изместване и добавяне на `<eos>`. Скоро ще видим защо ни трябват еднократно кодирани символи

Въпреки това, тази функция е **Pythonic**, т.е. тя не може автоматично да бъде преведена в изчислителен граф на Tensorflow. Ще получим грешки, ако се опитаме да използваме тази функция директно в `Dataset.map` функцията. Трябва да обгърнем този Pythonic повик, като използваме обвивката `py_function`:


In [6]:
def title_batch_fn(x):
    x = x['title']
    a,b = tf.py_function(title_batch,inp=[x],Tout=(tf.float32,tf.float32))
    return a,b

> **Забележка**: Разграничаването между Pythonic и Tensorflow функции за трансформация може да изглежда твърде сложно, и може би се питате защо не трансформираме набора от данни, използвайки стандартни Python функции, преди да го подадем на `fit`. Въпреки че това определено е възможно, използването на `Dataset.map` има огромно предимство, защото тръбопроводът за трансформация на данни се изпълнява чрез изчислителния граф на Tensorflow, който се възползва от изчисленията на GPU и минимизира нуждата от прехвърляне на данни между CPU/GPU.

Сега можем да изградим нашата генераторна мрежа и да започнем обучението. Тя може да бъде базирана на всяка рекурентна клетка, която обсъдихме в предишния модул (обикновена, LSTM или GRU). В нашия пример ще използваме LSTM.

Тъй като мрежата приема символи като вход, а размерът на речника е сравнително малък, нямаме нужда от embedding слой – входът, кодиран като one-hot, може директно да бъде подаден в LSTM клетката. Изходният слой ще бъде `Dense` класификатор, който ще преобразува изхода на LSTM в числа, кодирани като one-hot.

Освен това, тъй като работим с последователности с променлива дължина, можем да използваме слой `Masking`, за да създадем маска, която ще игнорира допълнителната част на низа. Това не е строго необходимо, защото не сме особено заинтересовани от всичко, което е след `<eos>` токена, но ще го използваме, за да натрупаме опит с този тип слой. `input_shape` ще бъде `(None, vocab_size)`, където `None` указва последователност с променлива дължина, а изходната форма също е `(None, vocab_size)`, както можете да видите от `summary`:


In [7]:
model = keras.models.Sequential([
    keras.layers.Masking(input_shape=(None,vocab_size)),
    keras.layers.LSTM(128,return_sequences=True),
    keras.layers.Dense(vocab_size,activation='softmax')
])

model.summary()
model.compile(loss='categorical_crossentropy')

model.fit(ds_train.batch(8).map(title_batch_fn))

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
masking (Masking)            (None, None, 84)          0         
_________________________________________________________________
lstm (LSTM)                  (None, None, 128)         109056    
_________________________________________________________________
dense (Dense)                (None, None, 84)          10836     
Total params: 119,892
Trainable params: 119,892
Non-trainable params: 0
_________________________________________________________________
15000/15000 [==============================] - 229s 15ms/step - loss: 1.5385


## Генериране на изход

Сега, след като сме обучили модела, искаме да го използваме за генериране на изход. На първо място, ни е необходим начин за декодиране на текст, представен чрез последователност от номера на токени. За тази цел можем да използваме функцията `tokenizer.sequences_to_texts`; обаче тя не работи добре с токенизация на ниво символи. Затова ще вземем речник на токените от токенизатора (наречен `word_index`), ще изградим обратна карта и ще напишем собствена функция за декодиране:


In [10]:
reverse_map = {val:key for key, val in tokenizer.word_index.items()}

def decode(x):
    return ''.join([reverse_map[t] for t in x])

Сега, нека започнем с генерирането. Ще започнем с някакъв низ `start`, ще го кодираме в последователност `inp`, и след това на всяка стъпка ще извикваме нашата мрежа, за да предскажем следващия символ.

Изходът на мрежата `out` е вектор с `vocab_size` елементи, представляващи вероятностите за всеки токен, и можем да намерим най-вероятния номер на токен, използвайки `argmax`. След това добавяме този символ към генерирания списък с токени и продължаваме с генерирането. Този процес на генериране на един символ се повтаря `size` пъти, за да се генерира необходимият брой символи, и приключваме по-рано, когато се срещне `eos_token`.


In [12]:
def generate(model,size=100,start='Today '):
        inp = tokenizer.texts_to_sequences([start])[0]
        chars = inp
        for i in range(size):
            out = model(tf.expand_dims(tf.one_hot(inp,vocab_size),0))[0][-1]
            nc = tf.argmax(out)
            if nc==eos_token:
                break
            chars.append(nc.numpy())
            inp = inp+[nc]
        return decode(chars)
    
generate(model)

'Today #39;s lead to strike for the strike for the strike for the strike (AFP)'

## Извличане на примерен изход по време на обучение

Тъй като нямаме полезни метрики като *точност*, единственият начин да видим дали нашият модел се подобрява, е чрез **извличане на примерен** генериран низ по време на обучението. За да направим това, ще използваме **callbacks**, т.е. функции, които можем да предадем на функцията `fit` и които ще бъдат извиквани периодично по време на обучението.


In [13]:
sampling_callback = keras.callbacks.LambdaCallback(
  on_epoch_end = lambda batch, logs: print(generate(model))
)

model.fit(ds_train.batch(8).map(title_batch_fn),callbacks=[sampling_callback],epochs=3)

Epoch 1/3
15000/15000 [==============================] - 226s 15ms/step - loss: 1.2703
Today #39;s a lead in the company for the strike
Epoch 2/3
15000/15000 [==============================] - 227s 15ms/step - loss: 1.2057
Today #39;s the Market Service on Security Start (AP)
Epoch 3/3
15000/15000 [==============================] - 226s 15ms/step - loss: 1.1752
Today #39;s a line on the strike to start for the start


Този пример вече генерира доста добър текст, но може да бъде подобрен по няколко начина:

* **Повече текст**. Използвахме само заглавия за нашата задача, но може да опитате с пълен текст. Имайте предвид, че RNNs не се справят добре с обработката на дълги последователности, затова има смисъл или да ги разделите на по-кратки изречения, или винаги да тренирате с фиксирана дължина на последователността с предварително зададена стойност `num_chars` (например, 256). Можете да опитате да промените горния пример в такава архитектура, използвайки [официалния урок на Keras](https://keras.io/examples/generative/lstm_character_level_text_generation/) като вдъхновение.

* **Многослойна LSTM**. Има смисъл да опитате с 2 или 3 слоя LSTM клетки. Както споменахме в предишния модул, всеки слой на LSTM извлича определени модели от текста, и в случай на генератор на ниво символи можем да очакваме, че по-ниският слой на LSTM ще бъде отговорен за извличането на срички, а по-високите слоеве - за думи и комбинации от думи. Това може лесно да се реализира чрез подаване на параметър за броя на слоевете към конструктора на LSTM.

* Може също да искате да експериментирате с **GRU единици** и да видите кои се представят по-добре, както и с **различни размери на скритите слоеве**. Твърде голям скрит слой може да доведе до прекомерно напасване (например, мрежата ще научи точния текст), а по-малък размер може да не произведе добри резултати.


## Генериране на мек текст и температура

В предишното определение на `generate`, винаги избирахме символа с най-висока вероятност като следващ символ в генерирания текст. Това водеше до факта, че текстът често "циклираше" между едни и същи последователности от символи отново и отново, както в този пример:
```
today of the second the company and a second the company ...
```

Но ако погледнем разпределението на вероятностите за следващия символ, може да се окаже, че разликата между няколко от най-високите вероятности не е голяма, например един символ може да има вероятност 0.2, а друг - 0.19 и т.н. Например, когато търсим следващия символ в последователността '*play*', следващият символ може еднакво добре да бъде интервал или **e** (както в думата *player*).

Това ни води до извода, че не винаги е "справедливо" да избираме символа с по-висока вероятност, защото изборът на втория по вероятност също може да доведе до смислен текст. По-разумно е да **вземем проба** от разпределението на вероятностите, дадено от изхода на мрежата.

Това вземане на проба може да се извърши с помощта на функцията `np.multinomial`, която реализира така нареченото **мултиномиално разпределение**. Функция, която реализира това **меко** генериране на текст, е дефинирана по-долу:


In [33]:
def generate_soft(model,size=100,start='Today ',temperature=1.0):
        inp = tokenizer.texts_to_sequences([start])[0]
        chars = inp
        for i in range(size):
            out = model(tf.expand_dims(tf.one_hot(inp,vocab_size),0))[0][-1]
            probs = tf.exp(tf.math.log(out)/temperature).numpy().astype(np.float64)
            probs = probs/np.sum(probs)
            nc = np.argmax(np.random.multinomial(1,probs,1))
            if nc==eos_token:
                break
            chars.append(nc)
            inp = inp+[nc]
        return decode(chars)

words = ['Today ','On Sunday ','Moscow, ','President ','Little red riding hood ']
    
for i in [0.3,0.8,1.0,1.3,1.8]:
    print(f"\n--- Temperature = {i}")
    for j in range(5):
        print(generate_soft(model,size=300,start=words[j],temperature=i))


--- Temperature = 0.3
Today #39;s strike #39; to start at the store return
On Sunday PO to Be Data Profit Up (Reuters)
Moscow, SP wins straight to the Microsoft #39;s control of the space start
President olding of the blast start for the strike to pay &lt;b&gt;...&lt;/b&gt;
Little red riding hood ficed to the spam countered in European &lt;b&gt;...&lt;/b&gt;

--- Temperature = 0.8
Today countie strikes ryder missile faces food market blut
On Sunday collores lose-toppy of sale of Bullment in &lt;b&gt;...&lt;/b&gt;
Moscow, IBM Diffeiting in Afghan Software Hotels (Reuters)
President Ol Luster for Profit Peaced Raised (AP)
Little red riding hood dace on depart talks #39; bank up

--- Temperature = 1.0
Today wits House buiting debate fixes #39; supervice stake again
On Sunday arling digital poaching In for level
Moscow, DS Up 7, Top Proble Protest Caprey Mamarian Strike
President teps help of roubler stepted lessabul-Dhalitics (AFP)
Little red riding hood signs on cash in Carter-youb

---

KeyError: 0

Въведохме още един параметър, наречен **температура**, който се използва, за да покаже колко строго трябва да се придържаме към най-високата вероятност. Ако температурата е 1.0, правим честно мултиномиално извадково вземане, а когато температурата достигне до безкрайност - всички вероятности стават равни и случайно избираме следващия символ. В примера по-долу можем да наблюдаваме, че текстът става безсмислен, когато увеличим температурата твърде много, и прилича на "циклично" твърдо генериран текст, когато се доближи до 0.



---

**Отказ от отговорност**:  
Този документ е преведен с помощта на AI услуга за превод [Co-op Translator](https://github.com/Azure/co-op-translator). Въпреки че се стремим към точност, моля, имайте предвид, че автоматизираните преводи може да съдържат грешки или неточности. Оригиналният документ на неговия роден език трябва да се счита за авторитетен източник. За критична информация се препоръчва професионален човешки превод. Ние не носим отговорност за каквито и да е недоразумения или погрешни интерпретации, произтичащи от използването на този превод.
